# NF264- Project 2: Digit recognizer

We  are  working  for  a  small  company  that  provides  machine learning  solutions  for  its  customers.   The  postal  office  needs an  AI system  to  automatically  deliver  mail.   As  a  part  of  the  system,  they  need a computer  program  that  recognises  handwritten  digits. We are providing  this  program  and  as machine  learning  experts, we write  the code that produces a classifier and this report that describes what we have done.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf 
from sklearnex import patch_sklearn
patch_sklearn()

C:\Users\elias\Miniconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\elias\Miniconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\elias\Miniconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

C:\Users\elias\Miniconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be unders

## The Dataset
The MNIST dataset consist of 70000 images of handwritten digits. Each image consist of a 28x28 pixel images with a grayscale value between 0-255. They are given as a list of 70000 with each list having length 28x28 = 784. Which is confirmed by the shape printed below.

In [2]:
X = pd.read_csv('handwritten_digits_images.csv', header=None).to_numpy()
y = pd.read_csv('handwritten_digits_labels.csv', header=None).to_numpy()
print(X.shape)

(70000, 784)


## Preprocessing steps
The labels of each images is represented as a digit between 0 and 9. We can make this label categorical, meaning they are all represented the same way as a bit array with 10 elements, where for example 4 is a 1 at the 5th index.

We also want to normalize the grayscale values from 0-255 to 0-1

In [3]:
from keras.utils import to_categorical

y = to_categorical(y)

# Normalize to range 0-1
X = X.astype('float32')
X = X / 255.0

Using TensorFlow backend.


Here is an example of a image and its corresponding label.

In [4]:
print(y[15000])

px.imshow(X[15000].reshape(28,28), color_continuous_scale=["white", "black"])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


## Splitting data
We split the data in 80% training data, 10% validation data used for evaluating and tuning hyperparameters, and 10% unseen test data which is used to choose the best model.

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

print('Train: X=%s, y=%s' % (X_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (X_test.shape, y_test.shape))
print('Validate: X=%s, y=%s' % (X_val.shape, y_val.shape))

Train: X=(56000, 784), y=(56000, 10)
Test: X=(7000, 784), y=(7000, 10)
Validate: X=(7000, 784), y=(7000, 10)


## Candidate  algorithms  and  choice  of  candidate  hyperparameters  (and why were the others left out)

We want to chose a classifier algorithm since a classifier utilizes some training data to understand how given input variables relate to a class. In this case, pictures of integers 0-9 are used as the training data. When the classifier is trained accurately, it can be used to detect integers for the Postal office. There are many candidates in this space. 

Firstly, we choose K Nearest Neighbors Classifier since this is kind of a baseline model, and we have implemnted this model from scratch is previous courses so we are well aware of the algorithm.

Secondly, we want to test a descision tree classifier since we know that this is an effective Classifier from our previous Project, where we classified a dataset with 10 features. When we think about a writtin digit, there are probably some descisions thart could be made in a descision tree, such as if it has a single line in vertical direction it is a 1 or 7, or if it contains two circles it is a 8. From the pixel data we ecpect there will be some kind of denominator that could classify the image into a category of digits.

Lastly, we  want to explore a Sequential Convolutional Neural Network Classifier since we are not as familier with this tool, and want to learn more about implemtning this Classifier. Neural Nets can be very powerfull if trained accuratly, so we want to explore if this could be a feasible solution for recognizing digits.

## Chosen performance measure
When chosing perfomance measure there are several that could be used, i.e MSE and RMSE, but we want to use the accuracy in percentage (0-100%) on the test data for model selection, and the accuracy on validation data for model evaluation.

## K Nearest Neighbors Classifier
The K-nearest neighbors (KNN) algorithm is a data classification method for estimating the probability that a data point will become a member of one or another group based on which group the data points are closest to it. A classification problem has a discrete value as its output. It is a type of supervised machine learning algorithm used to solve classification (and regression) problems. The algorithm is also called a lazy learning and non-parametic algorithm. This is because it is lazy and does not preform any training when you supply the training data. It just stores the data during the training time and does not perform any calculations. The algorithm does not build a model until a query is performed on the data set. It is considered a non-parametric methods because it does not make any assumptions about the underlying data distribution. It also involves classifying a data point by looking at the nearest annotated data point.

A advantage of using it, is that the training phase of K-nearest neighbor classification is much faster compared to other classification algorithms. There is no need to train a model for generalization, that is why KNN is known as the simple and instance-based learning algorithm. One disadvantage of using KNN is that the testing phase of K-nearest neighbor classification is slower and costlier in terms of time and memory. It requires large memory for storing the entire training dataset for prediction.

### Hyperparameters
We tune the number of nearest neighbors k to asses for choosing the label of the each image.

In [6]:
from sklearn.neighbors import KNeighborsClassifier

kVals = [1, 2, 3, 4, 5, 6, 7, 8 , 9, 10, 15, 30]
accuracies = []

for k in kVals: # Testing many k hyperparametyers to optimize performance
	model = KNeighborsClassifier(algorithm='auto', n_neighbors=k)
	model.fit(X_train, y_train)

	score = model.score(X_val, y_val)
	print("k=%d, validation accuracy=%.2f%%" % (k, score * 100))
	accuracies.append([k, score * 100])

k=1, validation accuracy=97.01%
k=2, validation accuracy=94.61%
k=3, validation accuracy=96.86%
k=4, validation accuracy=95.73%
k=5, validation accuracy=96.66%
k=6, validation accuracy=95.53%
k=7, validation accuracy=96.36%
k=8, validation accuracy=95.54%
k=9, validation accuracy=96.30%
k=10, validation accuracy=95.50%
k=15, validation accuracy=95.60%
k=30, validation accuracy=94.11%


In [7]:
#Plotting data
df = pd.DataFrame(accuracies, columns = ['k', 'Accuracy'])
px.line(df, x="k", y = 'Accuracy', title="kNN Model accuracy on validation data")

In [8]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

model = KNeighborsClassifier(n_neighbors=1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print("Evaluation of test data")
print(classification_report(y_test, predictions))
print("sklearn KNeighborsClassifier Test data accuracy: {:3.2f}%".format(accuracy_score(y_test, predictions)*100))

Evaluation of test data
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       685
           1       0.97      0.99      0.98       778
           2       0.98      0.97      0.98       671
           3       0.97      0.96      0.96       690
           4       0.98      0.97      0.98       733
           5       0.96      0.96      0.96       644
           6       0.98      0.98      0.98       729
           7       0.96      0.97      0.97       694
           8       0.99      0.94      0.96       670
           9       0.96      0.97      0.96       706

   micro avg       0.97      0.97      0.97      7000
   macro avg       0.97      0.97      0.97      7000
weighted avg       0.97      0.97      0.97      7000
 samples avg       0.97      0.97      0.97      7000

sklearn KNeighborsClassifier Test data accuracy: 97.14%


### kNN Findings
We found best results with k=1 which gives us a accuracy on test of 97,14%

## Decision Tree Classifier
Decision tree is a supervised machine learning algorithm that uses a set of rules to make decisions. A decision tree has a flowchart-like tree structure where an internal node represents feature, the branch represents a decision rule and each leaf node represents the outcome. 

The most important feature is the capability of capturing descriptive decisionmaking knowledge from the supplied data. A decision tree can be generated from training sets. Decision tree classifier generates the actual prediction at the leaf nodes, more information can be stored at the leaf nodes. The algorithm is a distribution-free or non-parametric method, which does not depend upon probability distribution assumptions. It can handle high dimensional data with good accuracy. 


### Hyperparameters
As implemented in project 1 there are mainly 2 parameters to tweak. The impurity measure gini or entropy, and the max depth of the tree.

In [9]:
from sklearn.tree import DecisionTreeClassifier

criterion = ['gini','entropy']
max_depth = list(range(3, 31))

df = []
for d in max_depth:
    for c in criterion:
        dtc = DecisionTreeClassifier(criterion=c, max_depth=d)
        dtc.fit(X_train, y_train)
        acc = accuracy_score(y_val, dtc.predict(X_val))*100
        df.append([d,c,acc])
        print("Descicion tree with max depth: "+str(d) + ", impurity measure: "+c+", Accuracy: {:3.2f}%".format(acc))

Descicion tree with max depth: 3, impurity measure: gini, Accuracy: 24.13%
Descicion tree with max depth: 3, impurity measure: entropy, Accuracy: 30.99%
Descicion tree with max depth: 4, impurity measure: gini, Accuracy: 40.80%
Descicion tree with max depth: 4, impurity measure: entropy, Accuracy: 46.44%
Descicion tree with max depth: 5, impurity measure: gini, Accuracy: 52.33%
Descicion tree with max depth: 5, impurity measure: entropy, Accuracy: 59.91%
Descicion tree with max depth: 6, impurity measure: gini, Accuracy: 66.29%
Descicion tree with max depth: 6, impurity measure: entropy, Accuracy: 68.49%
Descicion tree with max depth: 7, impurity measure: gini, Accuracy: 70.57%
Descicion tree with max depth: 7, impurity measure: entropy, Accuracy: 73.99%
Descicion tree with max depth: 8, impurity measure: gini, Accuracy: 75.94%
Descicion tree with max depth: 8, impurity measure: entropy, Accuracy: 78.09%
Descicion tree with max depth: 9, impurity measure: gini, Accuracy: 80.60%
Descici

In [10]:
#Plotting data
df = pd.DataFrame(df, columns = ['Depth', 'Impurity', 'Accuracy'])
px.line(df, x="Depth", y = 'Accuracy', color='Impurity', title="DecisionTreeClassifier accuracy on validation data")

We chose the best hyperparameters based on the graph above.

In [11]:
DecisionTree = DecisionTreeClassifier()
DecisionTree.fit(X_train, y_train)
predictions = DecisionTree.predict(X_test)

print("Evaluation of test data")
print(classification_report(y_test, predictions))

print("sklearn DecisionTreeClassifier Test data accuracy: {:3.2f}%".format(accuracy_score(y_test, predictions)*100))

Evaluation of test data
              precision    recall  f1-score   support

           0       0.92      0.91      0.92       685
           1       0.95      0.96      0.95       778
           2       0.84      0.87      0.86       671
           3       0.84      0.86      0.85       690
           4       0.89      0.88      0.88       733
           5       0.80      0.81      0.81       644
           6       0.90      0.91      0.90       729
           7       0.90      0.92      0.91       694
           8       0.83      0.78      0.80       670
           9       0.84      0.82      0.83       706

   micro avg       0.87      0.87      0.87      7000
   macro avg       0.87      0.87      0.87      7000
weighted avg       0.87      0.87      0.87      7000
 samples avg       0.87      0.87      0.87      7000

sklearn DecisionTreeClassifier Test data accuracy: 87.39%


We found best results with impurity measure entropy and depth of 30 which gives us a accuracy on test of about 87%. Given the kNN baseline performed 97% accuracy we can discard this model since it does not perform better.

## Sequential Convolutional Neural Network Classifier
Convolutional networks (CNN) are a specialized type of neural networks that use convolution in place of general matrix multiplication in at least one of their layers. CNNs consists of an input layer, hidden layers and a output layer. The hidden layers are called hidden, because their inputs and outputs are masked by the activation function and final convolution. This usually means that it includes a layer that preforms a dot product of the convolution kernel with the layers input matrix.

CNNs use relatively little pre-processing compared to other image classification algorithms, which means that the networks learns to optimize the filters (or kernel) trough automated learning, while in traditional algorithms these filters are hand-engineered.
CNNs take advantage of the hierarchial pattern in data. It assemble patterns of increasing complexity unsing smaller and simpler patterns embossed in their filters. Convolutional networks are a specialized type of neural networks that use convolution in place of general matrix multiplication in at least one of their layers.


### Hyperparameters

Layers
Activation function
keras.optimizers import SGD

In [12]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.optimizers import Adam
from sklearn.model_selection import KFold
from keras.layers import BatchNormalization

print(tf.__version__)

if tf.test.gpu_device_name(): 
    print('GPU Device:{}'.format(tf.test.gpu_device_name()))

1.10.0
GPU Device:/device:GPU:0


We need to reshape the image data for the nural network to accept it as a single color channel.

In [13]:
X = X.reshape(X.shape[0], 28, 28, 1)
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

Define Model

Next, we need to define a baseline convolutional neural network model for the problem.

The model has two main aspects: the feature extraction front end comprised of convolutional and pooling layers, and the classifier backend that will make a prediction.

For the convolutional front-end, we can start with a single convolutional layer with a small filter size (3,3) and a modest number of filters (32) followed by a max pooling layer. The filter maps can then be flattened to provide features to the classifier.

Given that the problem is a multi-class classification task, we know that we will require an output layer with 10 nodes in order to predict the probability distribution of an image belonging to each of the 10 classes. This will also require the use of a softmax activation function. Between the feature extractor and the output layer, we can add a dense layer to interpret the features, in this case with 100 nodes.

All layers will use the ReLU activation function and the He weight initialization scheme, both best practices.

We will use a conservative configuration for the stochastic gradient descent optimizer with a learning rate of 0.01 and a momentum of 0.9. The categorical cross-entropy loss function will be optimized, suitable for multi-class classification, and we will monitor the classification accuracy metric, which is appropriate given we have the same number of examples in each of the 10 classes.

In [14]:
def neural_net_1(X_train, y_train, X_val, y_val, X_test, y_test, epochs = 5):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax')) # Output layer of 10 integers
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

	model.fit(X_train, y_train, epochs=epochs, batch_size=64, validation_data=(X_val, y_val), verbose=1)
	_, acc = model.evaluate(X_test, y_test, verbose=1)
	print('Model accuracy on test data:  %.3f' % (acc * 100.0))
	
	return model
	
neural_net = neural_net_1(X_train, y_train, X_val, y_val, X_test, y_test, epochs=15)

Train on 56000 samples, validate on 7000 samples
Epoch 1/15
56000/56000 [==============================] - 13s 225us/step - loss: 0.2154 - acc: 0.9341 - val_loss: 0.1308 - val_acc: 0.9581
Epoch 2/15
56000/56000 [==============================] - 9s 167us/step - loss: 0.0838 - acc: 0.9752 - val_loss: 0.0886 - val_acc: 0.9734
Epoch 3/15
56000/56000 [==============================] - 9s 166us/step - loss: 0.0552 - acc: 0.9836 - val_loss: 0.0640 - val_acc: 0.9787
Epoch 4/15
56000/56000 [==============================] - 9s 166us/step - loss: 0.0405 - acc: 0.9878 - val_loss: 0.0571 - val_acc: 0.9811
Epoch 5/15
56000/56000 [==============================] - 9s 164us/step - loss: 0.0306 - acc: 0.9909 - val_loss: 0.0565 - val_acc: 0.9816
Epoch 6/15
56000/56000 [==============================] - 9s 166us/step - loss: 0.0247 - acc: 0.9925 - val_loss: 0.0577 - val_acc: 0.9817
Epoch 7/15
56000/56000 [==============================] - 9s 165us/step - loss: 0.0189 - acc: 0.9946 - val_loss: 0.0516 - 

## Increase in Model Depth

There are many ways to change the model configuration in order to explore improvements over the baseline model.

Two common approaches involve changing the capacity of the feature extraction part of the model or changing the capacity or function of the classifier part of the model. Perhaps the point of biggest influence is a change to the feature extractor.

We can increase the depth of the feature extractor part of the model, following a VGG-like pattern of adding more convolutional and pooling layers with the same sized filter, while increasing the number of filters. In this case, we will add a double convolutional layer with 64 filters each, followed by another max pooling layer.

The updated version of the define_model() function with this change is listed below.

In [15]:
def neural_net_2(X_train, y_train, X_val, y_val, X_test, y_test, epochs = 5):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

	model.fit(X_train, y_train, epochs=epochs, batch_size=64, validation_data=(X_val, y_val), verbose=1)
	_, acc = model.evaluate(X_test, y_test, verbose=1)
	print('Model accuracy on test data:  %.3f' % (acc * 100.0))

	return model

neural_net = neural_net_2(X_train, y_train, X_val, y_val, X_test, y_test, epochs=15)

Train on 56000 samples, validate on 7000 samples
Epoch 1/15
56000/56000 [==============================] - 13s 229us/step - loss: 0.1603 - acc: 0.9492 - val_loss: 0.0695 - val_acc: 0.9779
Epoch 2/15
56000/56000 [==============================] - 13s 224us/step - loss: 0.0550 - acc: 0.9832 - val_loss: 0.0506 - val_acc: 0.9833
Epoch 3/15
56000/56000 [==============================] - 13s 224us/step - loss: 0.0365 - acc: 0.9885 - val_loss: 0.0464 - val_acc: 0.9846
Epoch 4/15
56000/56000 [==============================] - 12s 219us/step - loss: 0.0294 - acc: 0.9906 - val_loss: 0.0474 - val_acc: 0.9870
Epoch 5/15
56000/56000 [==============================] - 12s 216us/step - loss: 0.0217 - acc: 0.9933 - val_loss: 0.0391 - val_acc: 0.9879
Epoch 6/15
56000/56000 [==============================] - 12s 214us/step - loss: 0.0176 - acc: 0.9943 - val_loss: 0.0464 - val_acc: 0.9870
Epoch 7/15
56000/56000 [==============================] - 12s 213us/step - loss: 0.0137 - acc: 0.9956 - val_loss: 0.0

## K-folds

In [16]:
def neural_net(X, y, epochs = 5, folds=3):
    scores = []
    hist = []
    kfold = KFold(folds, shuffle=True, random_state=1)
    
    for train_ix, val_ix in kfold.split(X):
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(10, activation='softmax'))
        opt = SGD(lr=0.01, momentum=0.9)
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

        trainX, trainY, valX, valY = X[train_ix], y[train_ix], X[val_ix], y[val_ix]
        model.fit(trainX, trainY, epochs=epochs, batch_size=32, validation_data=(valX, valY), verbose=1)
        
        _, acc = model.evaluate(valX, valY, verbose=1)
        print('> %.3f' % (acc * 100.0))
        
        scores.append(acc)
        hist.append(model)
        
    return scores, hist
    

final = neural_net(X,y, epochs=20, folds=5)

Train on 56000 samples, validate on 14000 samples
Epoch 1/20
56000/56000 [==============================] - 26s 456us/step - loss: 0.1218 - acc: 0.9606 - val_loss: 0.0484 - val_acc: 0.9851
Epoch 2/20
56000/56000 [==============================] - 25s 442us/step - loss: 0.0424 - acc: 0.9866 - val_loss: 0.0462 - val_acc: 0.9859
Epoch 3/20
56000/56000 [==============================] - 25s 443us/step - loss: 0.0297 - acc: 0.9904 - val_loss: 0.0320 - val_acc: 0.9894
Epoch 4/20
56000/56000 [==============================] - 25s 443us/step - loss: 0.0216 - acc: 0.9930 - val_loss: 0.0316 - val_acc: 0.9894
Epoch 5/20
56000/56000 [==============================] - 25s 449us/step - loss: 0.0157 - acc: 0.9949 - val_loss: 0.0375 - val_acc: 0.9886
Epoch 6/20
56000/56000 [==============================] - 25s 454us/step - loss: 0.0115 - acc: 0.9964 - val_loss: 0.0385 - val_acc: 0.9890
Epoch 7/20
56000/56000 [==============================] - 25s 450us/step - loss: 0.0103 - acc: 0.9966 - val_loss: 0.

In [17]:
print(final[0])
tests = list(range(1,len(final[0])+1))

fig = px.scatter(x = tests, y = final[0], labels=dict(x="K-folds test number:", y="Accuracy on validation data"))

fig.update_layout(xaxis={'tickformat': ',d'})
fig.update_layout(yaxis_range=[0,1])

[0.9924285714285714, 0.9921428571428571, 0.9920714285714286, 0.993, 0.9915]


By using kfolds and plotting the graph above we see that the data is not overfitted to the specific data-split and we can assume it is generilzed for unseen data. We now choose the best performing model and test it on unseen data.

In [18]:
best_model = final[1][final[0].index(max(final[0]))]

pred = best_model.predict_classes(X_test)
y_test_digits = [np.argmax(dig) for dig in y_test]
print(classification_report(y_test_digits, pred))

print("keras Sequential Test data accuracy: {:3.2f}%".format(accuracy_score(y_test_digits, pred)*100))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       685
           1       1.00      1.00      1.00       778
           2       1.00      1.00      1.00       671
           3       1.00      1.00      1.00       690
           4       1.00      1.00      1.00       733
           5       1.00      1.00      1.00       644
           6       1.00      1.00      1.00       729
           7       1.00      1.00      1.00       694
           8       1.00      1.00      1.00       670
           9       1.00      1.00      1.00       706

    accuracy                           1.00      7000
   macro avg       1.00      1.00      1.00      7000
weighted avg       1.00      1.00      1.00      7000

keras Sequential Test data accuracy: 99.87%


## What is your final classifier and how does it work.

We choose the Sequential neural net model as our final classifier with accuracy of 99.89%. Its details are described above
Neural net
## How  well  it  is  expected  to  perform  in  production  (on  unseen  data).Justify your estimate
We expect it to perform good by classifying almost all (99%) of unsees digits

## Measures taken to avoid overfitting
We tested the final model with folding data dataset to ensure that the split does not affect model performance. We also clearly splitted the set into train, validation and test which were used for each their part of assesing and improving the model.

## Given more resources (time or computing resources),  how would you improve your solution
More Epochs and K-folds if i had a stronger GPU. Also i had to use older versions of CUDA, CUDNN, Tenserflow and Keras to work with my GPU. Current versions of the library may be better optimized and produce better results. 

## Summary
should  give  a  short,  non-technical  overview  of  yourproject.  You should also argue,  based on your results,  whether or not themachine learning approach is appropriate for this task and what is your ex-pectation of its performance in real-life